### loading modules and data

In [1]:
########################################################

# If running with Google Colab

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
# # If using colab
from google.colab import files
files.upload()
# # select the 3 .py files (models, utils, data_utils)

Saving data_utils.py to data_utils.py
Saving models.py to models.py
Saving utils.py to utils.py


{'data_utils.py': b'import numpy as np\r\nimport scipy.signal as sig\r\nimport pywt\r\nfrom torch.utils.data import Dataset, DataLoader\r\nimport torch\r\n\r\n\'\'\'\r\nThe customized dataset and data augmentations are defined here.\r\n\r\n\'\'\'\r\nclass EEG_Dataset(Dataset):\r\n    \'\'\'\r\n    use use fold_idx to instantiate different train val datasets for k-fold cross validation\r\n\r\n    \'\'\'\r\n    def __init__ (self, X_train=None, y_train=None, p_train=None, X_val=None, y_val=None, p_val=None, X_test=None, y_test=None, p_test=None, mode=\'train\'):\r\n        if mode == \'train\':\r\n            self.X = X_train\r\n            self.y = y_train- 769\r\n            self.p = p_train\r\n            \r\n        elif mode == \'val\':\r\n            self.X = X_val\r\n            self.y = y_val- 769\r\n            self.p = p_val\r\n\r\n        elif mode == \'test\':\r\n            self.X = X_test\r\n            self.y = y_test - 769        \r\n            self.p = p_test\r\n\r\n   

In [3]:
########################################################

# If running with Google Colab
# Create a folder "C247" and then store the project datasets within that folder
# Check that your datasets are setup correctly

!ls "/content/gdrive/My Drive/C247" # File path

cwt_into_window_tests	__pycache__	     X_train_ICA10.npy
Eden_tests1.ipynb	X_test_filtered.npy  X_train_ICA15.npy
Eden_tests2.ipynb	X_test_ICA10.npy     X_train_ICA5.npy
EEG_loading.ipynb	X_test_ICA15.npy     X_train_ICA.npy
FinalProject		X_test_ICA5.npy      X_train_valid.npy
person_test.npy		X_test_ICA.npy	     y_test.npy
person_train_valid.npy	X_test.npy	     y_train_valid.npy


In [4]:
from models import *
from utils import *
from data_utils import *

GPU is available


In [5]:
# X_test = np.load("X_test.npy")
# y_test = np.load("y_test.npy")
# person_train_valid = np.load("person_train_valid.npy")
# X_train_valid = np.load("X_train_valid.npy")
# y_train_valid = np.load("y_train_valid.npy")
# person_test = np.load("person_test.npy")

# Change if your directory is different

# dataset_path = './data/' # Yiming Path
dataset_path = "/content/gdrive/My Drive/C247/" 

X_test = np.load(dataset_path + "X_test.npy")
y_test = np.load(dataset_path + "y_test.npy")
person_train_valid = np.load(dataset_path + "person_train_valid.npy")
X_train_valid = np.load(dataset_path + "X_train_valid.npy")
y_train_valid = np.load(dataset_path + "y_train_valid.npy")
person_test = np.load(dataset_path + "person_test.npy")
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))


Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


### CWT 0.1 & 0.4

In [0]:
# Uncomment for window
# train_fold, val_fold = Train_Val_Data(X_train_valid, y_train_valid)

# Uncomment for CWT -> Window
num_levels = 5
top_scale = 0.4
bottom_scale = 0.1
X_cwt, y_cwt, p_cwt = cwt_data2(X_train_valid, y_train_valid, person_train_valid, num_levels, bottom_scale = bottom_scale, top_scale=top_scale)
train_fold, val_fold = Train_Val_Data(X_cwt, y_cwt)
X_cwt[train_fold[0]].shape

(8460, 22, 1000)

In [0]:
# setting up the data augmentation here
aug_type = "window"
window_size = 200
stride = 50 # Make 20 normally
vote_num = 50 # Make 20 normally
num_folds = 1
num_epochs = 50
k = 0
dropout = 0.75

#drops = [0.5, 0.75]
learn_rates = [0.001, 0.0005, 0.0001]
lrn_rt = [0.0001]
X_train, y_train, p_train = X_cwt[train_fold[k]], y_cwt[train_fold[k]], p_cwt[train_fold[k]]
X_val, y_val, p_val = X_cwt[val_fold[k]], y_cwt[val_fold[k]], p_cwt[val_fold[k]]
X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_type, window_size=window_size, window_stride=stride)
if aug_type != 'window':
    X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, aug_type=aug_type)
EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)

for lrn_rt in lrn_rt:
    # indicate hyperparameters here
    model, criterion, optimizer = InitRNN(rnn_type='GRU', dropout=dropout, weight_decay=0, lr=lrn_rt)
    print ('lrn_rt {}'.format(lrn_rt))
    
    # Uncomment for just windowing
    # X_train, y_train, p_train = X_train_valid[train_fold[k]], y_train_valid[train_fold[k]], person_train_valid[train_fold[k]]
    # X_val, y_val, p_val = X_train_valid[val_fold[k]], y_train_valid[val_fold[k]], person_train_valid[val_fold[k]]

    # Uncomment for cwt
    best_model = TrainValRNN(model, criterion, optimizer, EEG_trainloader, EEG_valloader, num_epochs=num_epochs, aug_type=aug_type, window_size=window_size, vote_num=vote_num)


RNN TYPE: GRU
WEIGHT DECAY: 0
LEARNING RATE: 0.0001
lrn_rt 0.0001
epoch: 1      time: 31.88    loss: 1473.324    train acc: 0.253    val acc: 0.258
saving best model...
epoch: 2      time: 31.29    loss: 1466.368    train acc: 0.262    val acc: 0.283
saving best model...
epoch: 3      time: 31.58    loss: 1464.054    train acc: 0.269    val acc: 0.292
saving best model...
epoch: 4      time: 31.36    loss: 1461.840    train acc: 0.275    val acc: 0.310
saving best model...
epoch: 5      time: 31.48    loss: 1458.624    train acc: 0.283    val acc: 0.284
epoch: 6      time: 31.57    loss: 1455.232    train acc: 0.289    val acc: 0.316
saving best model...
epoch: 7      time: 31.70    loss: 1448.788    train acc: 0.300    val acc: 0.355
saving best model...
epoch: 8      time: 31.85    loss: 1439.505    train acc: 0.316    val acc: 0.346
epoch: 9      time: 31.98    loss: 1431.071    train acc: 0.326    val acc: 0.359
saving best model...
epoch: 10     time: 32.34    loss: 1421.392    tr

#### Training and Testing

In [0]:
# Uncomment if using cwt
X_cwt_test, y_cwt_test, person_cwt_test = cwt_data2(X_test, y_test, person_test, num_levels, top_scale=top_scale)

TestRNN(model, X_cwt_test, y_cwt_test, person_cwt_test, aug_type=aug_type, window_size=window_size, vote_num=vote_num)

Testing Accuracy: 0.5228


### CWT 0 & 0.4

In [0]:
# Uncomment for window
# train_fold, val_fold = Train_Val_Data(X_train_valid, y_train_valid)

# Uncomment for CWT -> Window
num_levels = 5
top_scale = 0.4
bottom_scale = 0
X_cwt, y_cwt, p_cwt = cwt_data2(X_train_valid, y_train_valid, person_train_valid, num_levels, bottom_scale = bottom_scale, top_scale=top_scale)
train_fold, val_fold = Train_Val_Data(X_cwt, y_cwt)
X_cwt[train_fold[0]].shape

(8460, 22, 1000)

In [0]:
# setting up the data augmentation here
aug_type = "window"
window_size = 200
stride = 50 # Make 20 normally
vote_num = 50 # Make 20 normally
num_folds = 1
num_epochs = 50
k = 0
dropout = 0.75

#drops = [0.5, 0.75]
learn_rates = [0.001, 0.0005, 0.0001]
lrn_rt = [0.0001]
X_train, y_train, p_train = X_cwt[train_fold[k]], y_cwt[train_fold[k]], p_cwt[train_fold[k]]
X_val, y_val, p_val = X_cwt[val_fold[k]], y_cwt[val_fold[k]], p_cwt[val_fold[k]]
X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_type, window_size=window_size, window_stride=stride)
if aug_type != 'window':
    X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, aug_type=aug_type)
EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)

for lrn_rt in lrn_rt:
    # indicate hyperparameters here
    model, criterion, optimizer = InitRNN(rnn_type='GRU', dropout=dropout, weight_decay=0, lr=lrn_rt)
    print ('lrn_rt {}'.format(lrn_rt))
    
    # Uncomment for just windowing
    # X_train, y_train, p_train = X_train_valid[train_fold[k]], y_train_valid[train_fold[k]], person_train_valid[train_fold[k]]
    # X_val, y_val, p_val = X_train_valid[val_fold[k]], y_train_valid[val_fold[k]], person_train_valid[val_fold[k]]

    # Uncomment for cwt
    best_model = TrainValRNN(model, criterion, optimizer, EEG_trainloader, EEG_valloader, num_epochs=num_epochs, aug_type=aug_type, window_size=window_size, vote_num=vote_num)


RNN TYPE: GRU
WEIGHT DECAY: 0
LEARNING RATE: 0.0001
lrn_rt 0.0001
epoch: 1      time: 32.41    loss: 1472.372    train acc: 0.253    val acc: 0.237
saving best model...
epoch: 2      time: 32.57    loss: 1466.791    train acc: 0.263    val acc: 0.278
saving best model...
epoch: 3      time: 32.59    loss: 1465.286    train acc: 0.266    val acc: 0.278
epoch: 4      time: 32.52    loss: 1463.403    train acc: 0.272    val acc: 0.284
saving best model...
epoch: 5      time: 32.59    loss: 1461.749    train acc: 0.277    val acc: 0.283
epoch: 6      time: 32.46    loss: 1460.383    train acc: 0.280    val acc: 0.271
epoch: 7      time: 32.52    loss: 1457.075    train acc: 0.287    val acc: 0.305
saving best model...
epoch: 8      time: 32.60    loss: 1452.549    train acc: 0.295    val acc: 0.344
saving best model...
epoch: 9      time: 32.91    loss: 1444.215    train acc: 0.309    val acc: 0.351
saving best model...
epoch: 10     time: 32.88    loss: 1436.817    train acc: 0.319    val

#### Training and Testing

In [0]:
# Uncomment if using cwt
X_cwt_test, y_cwt_test, person_cwt_test = cwt_data2(X_test, y_test, person_test, num_levels, top_scale=top_scale)

TestRNN(model, X_cwt_test, y_cwt_test, person_cwt_test, aug_type=aug_type, window_size=window_size, vote_num=vote_num)

Testing Accuracy: 0.5174


### CWT 0.1 & 0.5

In [0]:
# Uncomment for window
# train_fold, val_fold = Train_Val_Data(X_train_valid, y_train_valid)

# Uncomment for CWT -> Window
num_levels = 5
top_scale = 0.5
bottom_scale = 0.1
X_cwt, y_cwt, p_cwt = cwt_data2(X_train_valid, y_train_valid, person_train_valid, num_levels, bottom_scale = bottom_scale, top_scale=top_scale)
train_fold, val_fold = Train_Val_Data(X_cwt, y_cwt)
X_cwt[train_fold[0]].shape

(8460, 22, 1000)

In [0]:
# setting up the data augmentation here
aug_type = "window"
window_size = 200
stride = 50 # Make 20 normally
vote_num = 50 # Make 20 normally
num_folds = 1
num_epochs = 50
k = 0
dropout = 0.75

#drops = [0.5, 0.75]
learn_rates = [0.001, 0.0005, 0.0001]
lrn_rt = [0.0001]
X_train, y_train, p_train = X_cwt[train_fold[k]], y_cwt[train_fold[k]], p_cwt[train_fold[k]]
X_val, y_val, p_val = X_cwt[val_fold[k]], y_cwt[val_fold[k]], p_cwt[val_fold[k]]
X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_type, window_size=window_size, window_stride=stride)
if aug_type != 'window':
    X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, aug_type=aug_type)
EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)

for lrn_rt in lrn_rt:
    # indicate hyperparameters here
    model, criterion, optimizer = InitRNN(rnn_type='GRU', dropout=dropout, weight_decay=0, lr=lrn_rt)
    print ('lrn_rt {}'.format(lrn_rt))
    
    # Uncomment for just windowing
    # X_train, y_train, p_train = X_train_valid[train_fold[k]], y_train_valid[train_fold[k]], person_train_valid[train_fold[k]]
    # X_val, y_val, p_val = X_train_valid[val_fold[k]], y_train_valid[val_fold[k]], person_train_valid[val_fold[k]]

    # Uncomment for cwt
    best_model = TrainValRNN(model, criterion, optimizer, EEG_trainloader, EEG_valloader, num_epochs=num_epochs, aug_type=aug_type, window_size=window_size, vote_num=vote_num)


RNN TYPE: GRU
WEIGHT DECAY: 0
LEARNING RATE: 0.0001
lrn_rt 0.0001
epoch: 1      time: 32.65    loss: 1473.585    train acc: 0.257    val acc: 0.266
saving best model...
epoch: 2      time: 32.76    loss: 1465.842    train acc: 0.265    val acc: 0.254
epoch: 3      time: 32.67    loss: 1463.246    train acc: 0.272    val acc: 0.295
saving best model...
epoch: 4      time: 32.80    loss: 1461.614    train acc: 0.275    val acc: 0.281
epoch: 5      time: 32.79    loss: 1458.480    train acc: 0.284    val acc: 0.306
saving best model...
epoch: 6      time: 32.81    loss: 1456.493    train acc: 0.288    val acc: 0.305
epoch: 7      time: 32.67    loss: 1452.382    train acc: 0.297    val acc: 0.327
saving best model...
epoch: 8      time: 32.79    loss: 1446.039    train acc: 0.308    val acc: 0.368
saving best model...
epoch: 9      time: 32.88    loss: 1425.098    train acc: 0.337    val acc: 0.371
saving best model...
epoch: 10     time: 32.81    loss: 1401.239    train acc: 0.364    val

#### Training and Testing

In [0]:
# Uncomment if using cwt
X_cwt_test, y_cwt_test, person_cwt_test = cwt_data2(X_test, y_test, person_test, num_levels, top_scale=top_scale)

TestRNN(model, X_cwt_test, y_cwt_test, person_cwt_test, aug_type=aug_type, window_size=window_size, vote_num=vote_num)

Testing Accuracy: 0.5219


### CWT 0.2 & 0.5

In [0]:
# Uncomment for window
# train_fold, val_fold = Train_Val_Data(X_train_valid, y_train_valid)

# Uncomment for CWT -> Window
num_levels = 5
top_scale = 0.5
bottom_scale = 0.2
X_cwt, y_cwt, p_cwt = cwt_data2(X_train_valid, y_train_valid, person_train_valid, num_levels, bottom_scale = bottom_scale, top_scale=top_scale)
train_fold, val_fold = Train_Val_Data(X_cwt, y_cwt)
X_cwt[train_fold[0]].shape

(8460, 22, 1000)

In [0]:
# setting up the data augmentation here
aug_type = "window"
window_size = 200
stride = 50 # Make 20 normally
vote_num = 50 # Make 20 normally
num_folds = 1
num_epochs = 50
k = 0
dropout = 0.75

#drops = [0.5, 0.75]
learn_rates = [0.001, 0.0005, 0.0001]
lrn_rt = [0.0001]
X_train, y_train, p_train = X_cwt[train_fold[k]], y_cwt[train_fold[k]], p_cwt[train_fold[k]]
X_val, y_val, p_val = X_cwt[val_fold[k]], y_cwt[val_fold[k]], p_cwt[val_fold[k]]
X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_type, window_size=window_size, window_stride=stride)
if aug_type != 'window':
    X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, aug_type=aug_type)
EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)

for lrn_rt in lrn_rt:
    # indicate hyperparameters here
    model, criterion, optimizer = InitRNN(rnn_type='GRU', dropout=dropout, weight_decay=0, lr=lrn_rt)
    print ('lrn_rt {}'.format(lrn_rt))
    
    # Uncomment for just windowing
    # X_train, y_train, p_train = X_train_valid[train_fold[k]], y_train_valid[train_fold[k]], person_train_valid[train_fold[k]]
    # X_val, y_val, p_val = X_train_valid[val_fold[k]], y_train_valid[val_fold[k]], person_train_valid[val_fold[k]]

    # Uncomment for cwt
    best_model = TrainValRNN(model, criterion, optimizer, EEG_trainloader, EEG_valloader, num_epochs=num_epochs, aug_type=aug_type, window_size=window_size, vote_num=vote_num)


RNN TYPE: GRU
WEIGHT DECAY: 0
LEARNING RATE: 0.0001
lrn_rt 0.0001
epoch: 1      time: 32.23    loss: 1475.100    train acc: 0.252    val acc: 0.246
saving best model...
epoch: 2      time: 32.21    loss: 1467.159    train acc: 0.261    val acc: 0.261
saving best model...
epoch: 3      time: 32.30    loss: 1464.151    train acc: 0.270    val acc: 0.290
saving best model...
epoch: 4      time: 32.32    loss: 1460.917    train acc: 0.278    val acc: 0.316
saving best model...
epoch: 5      time: 32.60    loss: 1457.575    train acc: 0.286    val acc: 0.326
saving best model...
epoch: 6      time: 32.46    loss: 1452.075    train acc: 0.296    val acc: 0.339
saving best model...
epoch: 7      time: 32.31    loss: 1443.873    train acc: 0.310    val acc: 0.346
saving best model...
epoch: 8      time: 32.49    loss: 1433.359    train acc: 0.325    val acc: 0.346
epoch: 9      time: 32.40    loss: 1425.400    train acc: 0.335    val acc: 0.371
saving best model...
epoch: 10     time: 32.33   

#### Training and Testing

In [0]:
# Uncomment if using cwt
X_cwt_test, y_cwt_test, person_cwt_test = cwt_data2(X_test, y_test, person_test, num_levels, top_scale=top_scale)

TestRNN(model, X_cwt_test, y_cwt_test, person_cwt_test, aug_type=aug_type, window_size=window_size, vote_num=vote_num)

Testing Accuracy: 0.4876


### CWT 0 & 0.3

In [0]:
# Uncomment for window
# train_fold, val_fold = Train_Val_Data(X_train_valid, y_train_valid)

# Uncomment for CWT -> Window
num_levels = 5
top_scale = 0.3
bottom_scale = 0
X_cwt, y_cwt, p_cwt = cwt_data2(X_train_valid, y_train_valid, person_train_valid, num_levels, bottom_scale = bottom_scale, top_scale=top_scale)
train_fold, val_fold = Train_Val_Data(X_cwt, y_cwt)
X_cwt[train_fold[0]].shape

(8460, 22, 1000)

In [0]:
# setting up the data augmentation here
aug_type = "window"
window_size = 200
stride = 50 # Make 20 normally
vote_num = 50 # Make 20 normally
num_folds = 1
num_epochs = 50
k = 0
dropout = 0.75

#drops = [0.5, 0.75]
learn_rates = [0.001, 0.0005, 0.0001]
lrn_rt = [0.0001]
X_train, y_train, p_train = X_cwt[train_fold[k]], y_cwt[train_fold[k]], p_cwt[train_fold[k]]
X_val, y_val, p_val = X_cwt[val_fold[k]], y_cwt[val_fold[k]], p_cwt[val_fold[k]]
X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_type, window_size=window_size, window_stride=stride)
if aug_type != 'window':
    X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, aug_type=aug_type)
EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)

for lrn_rt in lrn_rt:
    # indicate hyperparameters here
    model, criterion, optimizer = InitRNN(rnn_type='GRU', dropout=dropout, weight_decay=0, lr=lrn_rt)
    print ('lrn_rt {}'.format(lrn_rt))
    
    # Uncomment for just windowing
    # X_train, y_train, p_train = X_train_valid[train_fold[k]], y_train_valid[train_fold[k]], person_train_valid[train_fold[k]]
    # X_val, y_val, p_val = X_train_valid[val_fold[k]], y_train_valid[val_fold[k]], person_train_valid[val_fold[k]]

    # Uncomment for cwt
    best_model = TrainValRNN(model, criterion, optimizer, EEG_trainloader, EEG_valloader, num_epochs=num_epochs, aug_type=aug_type, window_size=window_size, vote_num=vote_num)


RNN TYPE: GRU
WEIGHT DECAY: 0
LEARNING RATE: 0.0001
lrn_rt 0.0001
epoch: 1      time: 32.58    loss: 1471.761    train acc: 0.255    val acc: 0.262
saving best model...
epoch: 2      time: 32.74    loss: 1466.900    train acc: 0.261    val acc: 0.263
saving best model...
epoch: 3      time: 32.58    loss: 1465.307    train acc: 0.264    val acc: 0.286
saving best model...
epoch: 4      time: 32.65    loss: 1463.205    train acc: 0.271    val acc: 0.270
epoch: 5      time: 32.62    loss: 1461.465    train acc: 0.277    val acc: 0.261
epoch: 6      time: 32.61    loss: 1459.599    train acc: 0.282    val acc: 0.255
epoch: 7      time: 32.62    loss: 1456.054    train acc: 0.289    val acc: 0.302
saving best model...
epoch: 8      time: 32.56    loss: 1451.728    train acc: 0.297    val acc: 0.296
epoch: 9      time: 32.64    loss: 1440.580    train acc: 0.313    val acc: 0.322
saving best model...
epoch: 10     time: 32.58    loss: 1433.230    train acc: 0.324    val acc: 0.315
epoch: 11

#### Training and Testing

In [0]:
# Uncomment if using cwt
X_cwt_test, y_cwt_test, person_cwt_test = cwt_data2(X_test, y_test, person_test, num_levels, top_scale=top_scale)

TestRNN(model, X_cwt_test, y_cwt_test, person_cwt_test, aug_type=aug_type, window_size=window_size, vote_num=vote_num)

Testing Accuracy: 0.5210


### CWT 0.1 & 0.3

In [0]:
# Uncomment for window
# train_fold, val_fold = Train_Val_Data(X_train_valid, y_train_valid)

# Uncomment for CWT -> Window
num_levels = 5
top_scale = 0.3
bottom_scale = 0.1
X_cwt, y_cwt, p_cwt = cwt_data2(X_train_valid, y_train_valid, person_train_valid, num_levels, bottom_scale = bottom_scale, top_scale=top_scale)
train_fold, val_fold = Train_Val_Data(X_cwt, y_cwt)
X_cwt[train_fold[0]].shape

(8460, 22, 1000)

In [0]:
# setting up the data augmentation here
aug_type = "window"
window_size = 190
stride = 50 # Make 20 normally
vote_num = 50 # Make 20 normally
num_folds = 1
num_epochs = 50
k = 0
dropout = 0.75

#drops = [0.5, 0.75]
learn_rates = [0.001, 0.0005, 0.0001]
lrn_rt = [0.0001]
X_train, y_train, p_train = X_cwt[train_fold[k]], y_cwt[train_fold[k]], p_cwt[train_fold[k]]
X_val, y_val, p_val = X_cwt[val_fold[k]], y_cwt[val_fold[k]], p_cwt[val_fold[k]]
X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_type, window_size=window_size, window_stride=stride)
if aug_type != 'window':
    X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, aug_type=aug_type)
EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)

for lrn_rt in lrn_rt:
    # indicate hyperparameters here
    model, criterion, optimizer = InitRNN(rnn_type='GRU', dropout=dropout, weight_decay=0, lr=lrn_rt)
    print ('lrn_rt {}'.format(lrn_rt))
    
    # Uncomment for just windowing
    # X_train, y_train, p_train = X_train_valid[train_fold[k]], y_train_valid[train_fold[k]], person_train_valid[train_fold[k]]
    # X_val, y_val, p_val = X_train_valid[val_fold[k]], y_train_valid[val_fold[k]], person_train_valid[val_fold[k]]

    # Uncomment for cwt
    best_model = TrainValRNN(model, criterion, optimizer, EEG_trainloader, EEG_valloader, num_epochs=num_epochs, aug_type=aug_type, window_size=window_size, vote_num=vote_num)


RNN TYPE: GRU
WEIGHT DECAY: 0
LEARNING RATE: 0.0001
lrn_rt 0.0001
epoch: 1      time: 92.83    loss: 1472.780    train acc: 0.255    val acc: 0.250
saving best model...
epoch: 2      time: 81.85    loss: 1466.467    train acc: 0.263    val acc: 0.244
epoch: 3      time: 80.49    loss: 1463.676    train acc: 0.272    val acc: 0.255
saving best model...
epoch: 4      time: 80.74    loss: 1461.695    train acc: 0.278    val acc: 0.286
saving best model...
epoch: 5      time: 76.67    loss: 1458.708    train acc: 0.282    val acc: 0.293
saving best model...
epoch: 6      time: 77.29    loss: 1455.895    train acc: 0.288    val acc: 0.295
saving best model...
epoch: 7      time: 80.02    loss: 1451.225    train acc: 0.299    val acc: 0.306
saving best model...
epoch: 8      time: 80.44    loss: 1440.769    train acc: 0.314    val acc: 0.315
saving best model...
epoch: 9      time: 79.99    loss: 1428.534    train acc: 0.328    val acc: 0.358
saving best model...
epoch: 10     time: 79.60   

#### Training and Testing

In [0]:
# Uncomment if using cwt
X_cwt_test, y_cwt_test, person_cwt_test = cwt_data2(X_test, y_test, person_test, num_levels, top_scale=top_scale)

TestRNN(model, X_cwt_test, y_cwt_test, person_cwt_test, aug_type=aug_type, window_size=window_size, vote_num=vote_num)

RuntimeError: ignored

### CWT 0.2 & 0.4

In [0]:
# Uncomment for window
# train_fold, val_fold = Train_Val_Data(X_train_valid, y_train_valid)

# Uncomment for CWT -> Window
num_levels = 5
top_scale = 0.4
bottom_scale = 0.2
X_cwt, y_cwt, p_cwt = cwt_data2(X_train_valid, y_train_valid, person_train_valid, num_levels, bottom_scale = bottom_scale, top_scale=top_scale)
train_fold, val_fold = Train_Val_Data(X_cwt, y_cwt)
X_cwt[train_fold[0]].shape

(8460, 22, 1000)

In [0]:
# setting up the data augmentation here
aug_type = "window"
window_size = 200
stride = 50 # Make 20 normally
vote_num = 50 # Make 20 normally
num_folds = 1
num_epochs = 50
k = 0
dropout = 0.75

#drops = [0.5, 0.75]
learn_rates = [0.001, 0.0005, 0.0001]
lrn_rt = [0.0001]
X_train, y_train, p_train = X_cwt[train_fold[k]], y_cwt[train_fold[k]], p_cwt[train_fold[k]]
X_val, y_val, p_val = X_cwt[val_fold[k]], y_cwt[val_fold[k]], p_cwt[val_fold[k]]
X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_type, window_size=window_size, window_stride=stride)
if aug_type != 'window':
    X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, aug_type=aug_type)
EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)

for lrn_rt in lrn_rt:
    # indicate hyperparameters here
    model, criterion, optimizer = InitRNN(rnn_type='GRU', dropout=dropout, weight_decay=0, lr=lrn_rt)
    print ('lrn_rt {}'.format(lrn_rt))
    
    # Uncomment for just windowing
    # X_train, y_train, p_train = X_train_valid[train_fold[k]], y_train_valid[train_fold[k]], person_train_valid[train_fold[k]]
    # X_val, y_val, p_val = X_train_valid[val_fold[k]], y_train_valid[val_fold[k]], person_train_valid[val_fold[k]]

    # Uncomment for cwt
    best_model = TrainValRNN(model, criterion, optimizer, EEG_trainloader, EEG_valloader, num_epochs=num_epochs, aug_type=aug_type, window_size=window_size, vote_num=vote_num)


RNN TYPE: GRU
WEIGHT DECAY: 0
LEARNING RATE: 0.0001
lrn_rt 0.0001
epoch: 1      time: 33.69    loss: 1471.898    train acc: 0.256    val acc: 0.249
saving best model...
epoch: 2      time: 33.67    loss: 1465.754    train acc: 0.264    val acc: 0.268
saving best model...
epoch: 3      time: 33.47    loss: 1461.695    train acc: 0.275    val acc: 0.329
saving best model...
epoch: 4      time: 33.69    loss: 1457.330    train acc: 0.287    val acc: 0.330
saving best model...
epoch: 5      time: 33.75    loss: 1452.199    train acc: 0.296    val acc: 0.332
saving best model...
epoch: 6      time: 33.77    loss: 1441.770    train acc: 0.312    val acc: 0.355
saving best model...
epoch: 7      time: 33.69    loss: 1433.320    train acc: 0.322    val acc: 0.354
epoch: 8      time: 33.64    loss: 1425.350    train acc: 0.332    val acc: 0.401
saving best model...
epoch: 9      time: 33.61    loss: 1416.394    train acc: 0.342    val acc: 0.377
epoch: 10     time: 33.56    loss: 1398.971    tr

#### Training and Testing

In [0]:
# Uncomment if using cwt
X_cwt_test, y_cwt_test, person_cwt_test = cwt_data2(X_test, y_test, person_test, num_levels, top_scale=top_scale)

TestRNN(model, X_cwt_test, y_cwt_test, person_cwt_test, aug_type=aug_type, window_size=window_size, vote_num=vote_num)

Testing Accuracy: 0.4903


### CWT 0.2 & 0.3

In [0]:
# Uncomment for window
# train_fold, val_fold = Train_Val_Data(X_train_valid, y_train_valid)

# Uncomment for CWT -> Window
num_levels = 5
top_scale = 0.3
bottom_scale = 0.2
X_cwt, y_cwt, p_cwt = cwt_data2(X_train_valid, y_train_valid, person_train_valid, num_levels, bottom_scale = bottom_scale, top_scale=top_scale)
train_fold, val_fold = Train_Val_Data(X_cwt, y_cwt)
X_cwt[train_fold[0]].shape

(8460, 22, 1000)

In [0]:
# setting up the data augmentation here
aug_type = "window"
window_size = 200
stride = 50 # Make 20 normally
vote_num = 50 # Make 20 normally
num_folds = 1
num_epochs = 50
k = 0
dropout = 0.75

#drops = [0.5, 0.75]
learn_rates = [0.001, 0.0005, 0.0001]
lrn_rt = [0.0001]
X_train, y_train, p_train = X_cwt[train_fold[k]], y_cwt[train_fold[k]], p_cwt[train_fold[k]]
X_val, y_val, p_val = X_cwt[val_fold[k]], y_cwt[val_fold[k]], p_cwt[val_fold[k]]
X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_type, window_size=window_size, window_stride=stride)
if aug_type != 'window':
    X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, aug_type=aug_type)
EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)

for lrn_rt in lrn_rt:
    # indicate hyperparameters here
    model, criterion, optimizer = InitRNN(rnn_type='GRU', dropout=dropout, weight_decay=0, lr=lrn_rt)
    print ('lrn_rt {}'.format(lrn_rt))
    
    # Uncomment for just windowing
    # X_train, y_train, p_train = X_train_valid[train_fold[k]], y_train_valid[train_fold[k]], person_train_valid[train_fold[k]]
    # X_val, y_val, p_val = X_train_valid[val_fold[k]], y_train_valid[val_fold[k]], person_train_valid[val_fold[k]]

    # Uncomment for cwt
    best_model = TrainValRNN(model, criterion, optimizer, EEG_trainloader, EEG_valloader, num_epochs=num_epochs, aug_type=aug_type, window_size=window_size, vote_num=vote_num)


RNN TYPE: GRU
WEIGHT DECAY: 0
LEARNING RATE: 0.0001
lrn_rt 0.0001
epoch: 1      time: 32.89    loss: 1473.532    train acc: 0.255    val acc: 0.254
saving best model...
epoch: 2      time: 32.38    loss: 1466.318    train acc: 0.265    val acc: 0.270
saving best model...
epoch: 3      time: 32.36    loss: 1463.646    train acc: 0.270    val acc: 0.287
saving best model...
epoch: 4      time: 32.54    loss: 1460.814    train acc: 0.278    val acc: 0.315
saving best model...
epoch: 5      time: 32.31    loss: 1456.566    train acc: 0.290    val acc: 0.340
saving best model...
epoch: 6      time: 32.62    loss: 1451.076    train acc: 0.299    val acc: 0.347
saving best model...
epoch: 7      time: 32.56    loss: 1441.178    train acc: 0.315    val acc: 0.345
epoch: 8      time: 32.62    loss: 1422.182    train acc: 0.339    val acc: 0.372
saving best model...
epoch: 9      time: 32.69    loss: 1407.554    train acc: 0.353    val acc: 0.378
saving best model...
epoch: 10     time: 32.52   

#### Training and Testing

In [0]:
# Uncomment if using cwt
X_cwt_test, y_cwt_test, person_cwt_test = cwt_data2(X_test, y_test, person_test, num_levels, top_scale=top_scale)

TestRNN(model, X_cwt_test, y_cwt_test, person_cwt_test, aug_type=aug_type, window_size=window_size, vote_num=vote_num)

Testing Accuracy: 0.4777


### CWT 0.1 & 0.2

In [0]:
# Uncomment for window
# train_fold, val_fold = Train_Val_Data(X_train_valid, y_train_valid)

# Uncomment for CWT -> Window
num_levels = 5
top_scale = 0.2
bottom_scale = 0.1
X_cwt, y_cwt, p_cwt = cwt_data2(X_train_valid, y_train_valid, person_train_valid, num_levels, bottom_scale = bottom_scale, top_scale=top_scale)
train_fold, val_fold = Train_Val_Data(X_cwt, y_cwt)
X_cwt[train_fold[0]].shape

(8460, 22, 1000)

In [0]:
# setting up the data augmentation here
aug_type = "window"
window_size = 200
stride = 50 # Make 20 normally
vote_num = 50 # Make 20 normally
num_folds = 1
num_epochs = 50
k = 0
dropout = 0.75

#drops = [0.5, 0.75]
learn_rates = [0.001, 0.0005, 0.0001]
lrn_rt = [0.0001]
X_train, y_train, p_train = X_cwt[train_fold[k]], y_cwt[train_fold[k]], p_cwt[train_fold[k]]
X_val, y_val, p_val = X_cwt[val_fold[k]], y_cwt[val_fold[k]], p_cwt[val_fold[k]]
X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_type, window_size=window_size, window_stride=stride)
if aug_type != 'window':
    X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, aug_type=aug_type)
EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)

for lrn_rt in lrn_rt:
    # indicate hyperparameters here
    model, criterion, optimizer = InitRNN(rnn_type='GRU', dropout=dropout, weight_decay=0, lr=lrn_rt)
    print ('lrn_rt {}'.format(lrn_rt))
    
    # Uncomment for just windowing
    # X_train, y_train, p_train = X_train_valid[train_fold[k]], y_train_valid[train_fold[k]], person_train_valid[train_fold[k]]
    # X_val, y_val, p_val = X_train_valid[val_fold[k]], y_train_valid[val_fold[k]], person_train_valid[val_fold[k]]

    # Uncomment for cwt
    best_model = TrainValRNN(model, criterion, optimizer, EEG_trainloader, EEG_valloader, num_epochs=num_epochs, aug_type=aug_type, window_size=window_size, vote_num=vote_num)


RNN TYPE: GRU
WEIGHT DECAY: 0
LEARNING RATE: 0.0001
lrn_rt 0.0001
epoch: 1      time: 32.29    loss: 1471.897    train acc: 0.256    val acc: 0.247
saving best model...
epoch: 2      time: 32.39    loss: 1465.893    train acc: 0.262    val acc: 0.261
saving best model...
epoch: 3      time: 32.16    loss: 1463.173    train acc: 0.272    val acc: 0.282
saving best model...
epoch: 4      time: 32.35    loss: 1459.217    train acc: 0.282    val acc: 0.302
saving best model...
epoch: 5      time: 32.22    loss: 1453.787    train acc: 0.293    val acc: 0.309
saving best model...
epoch: 6      time: 32.36    loss: 1445.405    train acc: 0.307    val acc: 0.272
epoch: 7      time: 32.21    loss: 1430.902    train acc: 0.326    val acc: 0.356
saving best model...
epoch: 8      time: 32.37    loss: 1410.171    train acc: 0.344    val acc: 0.363
saving best model...
epoch: 9      time: 32.38    loss: 1394.141    train acc: 0.355    val acc: 0.381
saving best model...
epoch: 10     time: 32.29   

#### Training and Testing

In [0]:
# Uncomment if using cwt
X_cwt_test, y_cwt_test, person_cwt_test = cwt_data2(X_test, y_test, person_test, num_levels, top_scale=top_scale)

TestRNN(model, X_cwt_test, y_cwt_test, person_cwt_test, aug_type=aug_type, window_size=window_size, vote_num=vote_num)

Testing Accuracy: 0.4506


#ICA 15

In [0]:
X_train_ICA = np.load(dataset_path + "X_train_ICA15.npy")
X_test_ICA  = np.load(dataset_path + "X_test_ICA15.npy")

In [0]:
train_fold, val_fold = Train_Val_Data(X_train_ICA, y_train_valid)
X_train_ICA[train_fold[0]].shape

(1692, 15, 1000)

In [0]:
# setting up the data augmentation here
aug_type = "window"
window_size = 500
stride = 200 # Make 20 normally
vote_num = 50 # Make 20 normally
num_folds = 1
num_epochs = 100
k = 0
dropout = 0.25

#drops = [0.5, 0.75]
learn_rates = [0.001, 0.0005, 0.0001]
lrn_rt = [0.001]
X_train, y_train, p_train = X_train_ICA[train_fold[k]], y_train_valid[train_fold[k]], person_train_valid[train_fold[k]]
X_val, y_val, p_val = X_train_ICA[val_fold[k]], y_train_valid[val_fold[k]], person_train_valid[val_fold[k]]
X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_type, window_size=window_size, window_stride=stride)
if aug_type != 'window':
    X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, aug_type=aug_type)
EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)

for lrn_rt in lrn_rt:
    # indicate hyperparameters here
    model, criterion, optimizer = InitRNN(rnn_type='GRU', input_size=X_train.shape[1], dropout=dropout, weight_decay=0, lr=lrn_rt)
    print ('lrn_rt {}'.format(lrn_rt))
    
    # Uncomment for just windowing
    # X_train, y_train, p_train = X_train_valid[train_fold[k]], y_train_valid[train_fold[k]], person_train_valid[train_fold[k]]
    # X_val, y_val, p_val = X_train_valid[val_fold[k]], y_train_valid[val_fold[k]], person_train_valid[val_fold[k]]

    # Uncomment for cwt
    best_model = TrainValRNN(model, criterion, optimizer, EEG_trainloader, EEG_valloader, num_epochs=num_epochs, aug_type=aug_type, window_size=window_size, vote_num=vote_num)


RNN TYPE: GRU
WEIGHT DECAY: 0
LEARNING RATE: 0.001


KeyboardInterrupt: ignored

In [0]:
# Uncomment if using cwt
TestRNN(model, X_test_ICA, y_test, person_test, aug_type=aug_type, window_size=window_size, vote_num=vote_num)

Testing Accuracy: 0.2641


#ICA 5

In [0]:
X_train_ICA = np.load(dataset_path + "X_train_ICA5.npy")
X_test_ICA  = np.load(dataset_path + "X_test_ICA5.npy")

In [0]:
train_fold, val_fold = Train_Val_Data(X_train_ICA, y_train_valid)
X_train_ICA[train_fold[0]].shape

(1692, 5, 1000)

In [0]:
# setting up the data augmentation here
aug_type = "window"
window_size = 500
stride = 200 # Make 20 normally
vote_num = 50 # Make 20 normally
num_folds = 1
num_epochs = 100
k = 0
dropout = 0.25

#drops = [0.5, 0.75]
learn_rates = [0.001, 0.0005, 0.0001]
lrn_rt = [0.001]
X_train, y_train, p_train = X_train_ICA[train_fold[k]], y_train_valid[train_fold[k]], person_train_valid[train_fold[k]]
X_val, y_val, p_val = X_train_ICA[val_fold[k]], y_train_valid[val_fold[k]], person_train_valid[val_fold[k]]
X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_type, window_size=window_size, window_stride=stride)
if aug_type != 'window':
    X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, aug_type=aug_type)
EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)

for lrn_rt in lrn_rt:
    # indicate hyperparameters here
    model, criterion, optimizer = InitRNN(rnn_type='GRU', input_size=X_train.shape[1], dropout=dropout, weight_decay=0, lr=lrn_rt)
    print ('lrn_rt {}'.format(lrn_rt))
    
    # Uncomment for just windowing
    # X_train, y_train, p_train = X_train_valid[train_fold[k]], y_train_valid[train_fold[k]], person_train_valid[train_fold[k]]
    # X_val, y_val, p_val = X_train_valid[val_fold[k]], y_train_valid[val_fold[k]], person_train_valid[val_fold[k]]

    # Uncomment for cwt
    best_model = TrainValRNN(model, criterion, optimizer, EEG_trainloader, EEG_valloader, num_epochs=num_epochs, aug_type=aug_type, window_size=window_size, vote_num=vote_num)


RNN TYPE: GRU
WEIGHT DECAY: 0
LEARNING RATE: 0.001
lrn_rt 0.001
epoch: 1      time: 6.51    loss: 37.522    train acc: 0.245    val acc: 0.251
saving best model...
epoch: 2      time: 6.51    loss: 37.445    train acc: 0.237    val acc: 0.243
epoch: 3      time: 6.52    loss: 37.428    train acc: 0.252    val acc: 0.251
saving best model...
epoch: 4      time: 6.52    loss: 37.438    train acc: 0.260    val acc: 0.251
saving best model...
epoch: 5      time: 6.49    loss: 37.438    train acc: 0.259    val acc: 0.251
saving best model...
epoch: 6      time: 6.50    loss: 37.442    train acc: 0.251    val acc: 0.251
saving best model...
epoch: 7      time: 6.49    loss: 37.424    train acc: 0.259    val acc: 0.251
saving best model...
epoch: 8      time: 6.50    loss: 37.430    train acc: 0.254    val acc: 0.251
saving best model...
epoch: 9      time: 6.51    loss: 37.438    train acc: 0.260    val acc: 0.251
saving best model...
epoch: 10     time: 6.53    loss: 37.433    train acc: 0.

In [0]:
# Uncomment if using cwt
TestRNN(model, X_test_ICA, y_test, person_test, aug_type=aug_type, window_size=window_size, vote_num=vote_num)

Testing Accuracy: 0.2935


#ICA 10 BP filtered (30,50hz)

In [0]:
X_train_ICA = np.load(dataset_path + "X_train_ICA10.npy")
X_test_ICA  = np.load(dataset_path + "X_test_ICA10.npy")

In [7]:
train_fold, val_fold = Train_Val_Data(X_train_ICA, y_train_valid)
X_train_ICA[train_fold[0]].shape

(1692, 10, 1000)

In [0]:
# setting up the data augmentation here
aug_type = "window"
window_size = 200
stride = 50 # Make 20 normally
vote_num = 50 # Make 20 normally
num_folds = 1
num_epochs = 100
k = 0
dropout = 0.25

#drops = [0.5, 0.75]
learn_rates = [0.001, 0.0005, 0.0001]
lrn_rt = [0.001]
X_train, y_train, p_train = X_train_ICA[train_fold[k]], y_train_valid[train_fold[k]], person_train_valid[train_fold[k]]
X_val, y_val, p_val = X_train_ICA[val_fold[k]], y_train_valid[val_fold[k]], person_train_valid[val_fold[k]]
X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_type, window_size=window_size, window_stride=stride)
if aug_type != 'window':
    X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, aug_type=aug_type)
EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)

for lrn_rt in lrn_rt:
    # indicate hyperparameters here
    model, criterion, optimizer = InitRNN(rnn_type='GRU', input_size=X_train.shape[1], dropout=dropout, weight_decay=0, lr=lrn_rt)
    print ('lrn_rt {}'.format(lrn_rt))
    
    # Uncomment for just windowing
    # X_train, y_train, p_train = X_train_valid[train_fold[k]], y_train_valid[train_fold[k]], person_train_valid[train_fold[k]]
    # X_val, y_val, p_val = X_train_valid[val_fold[k]], y_train_valid[val_fold[k]], person_train_valid[val_fold[k]]

    # Uncomment for cwt
    best_model = TrainValRNN(model, criterion, optimizer, EEG_trainloader, EEG_valloader, num_epochs=num_epochs, aug_type=aug_type, window_size=window_size, vote_num=vote_num)


RNN TYPE: GRU
WEIGHT DECAY: 0
LEARNING RATE: 0.001
lrn_rt 0.001
epoch: 1      time: 7.01    loss: 294.036    train acc: 0.253    val acc: 0.243
saving best model...
epoch: 2      time: 6.98    loss: 293.933    train acc: 0.253    val acc: 0.251
saving best model...
epoch: 3      time: 7.01    loss: 293.878    train acc: 0.255    val acc: 0.255
saving best model...
epoch: 4      time: 7.02    loss: 293.876    train acc: 0.258    val acc: 0.251
epoch: 5      time: 6.98    loss: 293.893    train acc: 0.257    val acc: 0.251
epoch: 6      time: 6.99    loss: 293.884    train acc: 0.255    val acc: 0.251
epoch: 7      time: 6.96    loss: 293.851    train acc: 0.256    val acc: 0.251
epoch: 8      time: 6.97    loss: 293.868    train acc: 0.257    val acc: 0.267
saving best model...
epoch: 9      time: 6.96    loss: 293.831    train acc: 0.257    val acc: 0.243
epoch: 10     time: 6.99    loss: 293.812    train acc: 0.256    val acc: 0.251
epoch: 11     time: 7.00    loss: 293.804    train a

In [0]:
# Uncomment if using cwt
TestRNN(model, X_test_ICA, y_test, person_test, aug_type=aug_type, window_size=window_size, vote_num=vote_num)

RuntimeError: ignored

In [0]:
X_test_ICA.shape

(443, 5, 1000)